# Этап 5: LoRA (Low-Rank Adaptation) — Хирургическое дообучение

LoRA позволяет адаптировать модель под новую задачу, обучая менее 1% её параметров. 

### Математическая идея:
Мы представляем изменение весов $\Delta W$ как произведение двух матриц низкого ранга:
$$\Delta W = B \cdot A$$
Где $A \in \mathbb{R}^{r \times k}$ и $B \in \mathbb{R}^{d \times r}$, а ранг $r$ очень мал (например, 2, 4 или 8).

**Итоговый выход слоя:**
$$h = W x + \frac{\alpha}{r} (B A) x$$

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from src.model import GPTLanguageModel, device, decode, encode
import copy

# 1. Реализация LoRA-слоя для линейной проекции
class LoRALinear(nn.Module):
    def __init__(self, linear_layer, rank=4, alpha=8):
        super().__init__()
        self.linear = linear_layer
        self.rank = rank
        self.alpha = alpha
        
        # Замораживаем веса основного слоя
        self.linear.weight.requires_grad = False
        if self.linear.bias is not None:
            self.linear.bias.requires_grad = False
            
        # Инициализируем A и B матрицы
        in_features = self.linear.in_features
        out_features = self.linear.out_features
        
        self.lora_A = nn.Parameter(torch.randn(in_features, rank) * 0.01)
        self.lora_B = nn.Parameter(torch.zeros(rank, out_features))
        self.scaling = alpha / rank

    def forward(self, x):
        # Основной путь (замороженный)
        base_out = self.linear(x)
        
        # Путь LoRA (обучаемый)
        # (B, T, in) @ (in, rank) @ (rank, out)
        lora_out = (x @ self.lora_A) @ self.lora_B
        
        return base_out + lora_out * self.scaling

print("✅ LoRA-слой готов к использованию!")

Using device: mps
✅ LoRA-слой готов к использованию!


### 2. Внедрение LoRA в вашу модель
Мы заменим все слои `query` и `value` в блоках внимания на их LoRA-версии.

In [ ]:
def apply_lora(model, rank=4):
    # Итерируемся по компонентам модели
    for block in model.blocks:
        for head in block.sa.heads:
            # Заменяем query и value
            head.query = LoRALinear(head.query, rank=rank)
            head.value = LoRALinear(head.value, rank=rank)
    return model

base_model = GPTLanguageModel().to(device)
base_model.load_state_dict(torch.load('model_ckpt.pt', map_location=device))

lora_model = apply_lora(base_model, rank=4)

# Считаем параметры
total_params = sum(p.numel() for p in lora_model.parameters())
trainable_params = sum(p.numel() for p in lora_model.parameters() if p.requires_grad)

print(f"Всего параметров: {total_params:,}")
print(f"Обучаемых параметров (LoRA): {trainable_params:,}")
print(f"Доля обучаемых параметров: {100 * trainable_params / total_params:.2f}%")

### 3. Блиц-дообучение на новой задаче
Давайте заставим Шекспира выучить фразу про роботов и искусственный интеллект.

In [ ]:
fine_tune_text = "AI ROBOT: To be or not to be a machine, that is the computation! Machines will rule the code.\n" * 50
data_ft = torch.tensor(encode(fine_tune_text), dtype=torch.long, device=device)

# Простой цикл дообучения
optimizer = torch.optim.AdamW(lora_model.parameters(), lr=1e-3)
lora_model.train()

print("Начинаем LoRA-финтюнинг...")
for i in range(200):
    # Берем случайный кусок из текста про роботов
    ix = torch.randint(len(data_ft) - 256, (16,))
    x = torch.stack([data_ft[j:j+256] for j in ix])
    y = torch.stack([data_ft[j+1:j+256+1] for j in ix])
    
    logits, loss = lora_model(x, y)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    if i % 40 == 0:
        print(f"Шаг {i}, Loss: {loss.item():.4f}")

print("Дообучение завершено!")

### 4. Проверка результата
Теперь проверим, как наш Шекспир объединяет свои старые знания с новыми LoRA-адаптерами.

In [ ]:
lora_model.eval()
context = torch.tensor(encode("KING: "), dtype=torch.long, device=device).unsqueeze(0)
print("--- Output AFTER LoRA Fine-tuning ---")
print(decode(lora_model.generate(context, max_new_tokens=150)[0].tolist()))